In [ ]:
import numpy as np
import math
from copy import deepcopy
from typing import List, Callable, Union


In [293]:
def bin_to_float(bitstring: str) -> float:

    if len(bitstring) != 16 or not all(digit in '01' for digit in bitstring):
        raise ValueError("Invalid bitstring length")
    
    int_val = int(bitstring, 2)
    bytes_val = int_val.to_bytes(2, byteorder='big')

    return np.frombuffer(bytes_val, dtype='f2')[0]
    
print(bin_to_float('0000000000000001'))


1.526e-05


In [294]:
def decode_multidim(chromosome: str, dimensions: int) -> List[float]:
    
    if len(chromosome) % dimensions != 0:
        raise ValueError(f"Chromosome length must be divisible by dimensions ({dimensions})")
    
    bits_per_dim = len(chromosome) // dimensions
    
    if bits_per_dim != 16:
        raise ValueError("Each dimension must use 16 bits")
    
    values = []
    for i in range(dimensions):
        start_idx = i * bits_per_dim
        end_idx = start_idx + bits_per_dim
        dim_bits = chromosome[start_idx:end_idx]
        values.append(bin_to_float(dim_bits))
    
    return values


In [295]:
class Qubit():
    superposition_amplitude = 1 / np.sqrt(2)

    def __init__(self, measurement: bool) -> None:
        self.measurement: bool = measurement
        self.alpha: float = self.superposition_amplitude
        self.beta: float = self.superposition_amplitude
        self.superposition: bool = False
        
    def apply_hadamard(self) -> 'Qubit':
        superposition_amplitude = 1 / np.sqrt(2)

        self.alpha = superposition_amplitude
        self.beta = superposition_amplitude
        self.superposition = True
        
        return self
        
    def apply_xgate(self) -> None:
        self.measurement = not self.measurement
        
    def measure(self) -> bool:
        if not self.superposition:
            return self.measurement
        
        random_var = np.random.uniform(0, 1)
        self.measurement = np.pow(self.beta, 2) >= random_var
        self.superposition = False
        
        return self.measurement
    
    def __str__(self) -> str:
        return f'Qubit[superposition: {self.superposition}, [measurement: {self.measurement}]]'


In [296]:
def single_qubit_measurement(qubit: Qubit) -> bool:
    return qubit.measure()

def full_measurement(quantum_chromosome: List[Qubit]) -> str:
    for gene in quantum_chromosome:
        gene.measure()
        
    return ''.join(map(lambda x: str(int(x.measure())), quantum_chromosome))

def crossover(
        queen: List[Qubit],
        male: List[Qubit],
        num_swap_qubits: int
    ) -> str:

    quantum_chromosome = deepcopy(queen)
    random_ind = np.random.randint(0, len(queen) - num_swap_qubits)
    
    for i in range(random_ind, random_ind + num_swap_qubits):
        if queen[i].measure() == male[i].measure():
            quantum_chromosome[i].apply_hadamard()
        else:
            quantum_chromosome[i].apply_xgate()
    
    return full_measurement(quantum_chromosome)

In [297]:
def evolution(
        population_size: int,
        fitness: Callable[[Union[float, List[float]]], float],
        dimensions: int = 1,
        qubits_per_dim: int = 16,
        num_males: int = 20,
        num_elites: int = 20,
        max_iteration: int = 500,
        crossover_size: int = 3,
        maximize: bool = False
    ) -> Union[float, List[float]]:
    
    qubits_in_indiv = qubits_per_dim * dimensions
    population = []

    for i in range(population_size):
        quantum_chromosome = [Qubit(False).apply_hadamard() for _ in range(qubits_in_indiv)]
        quantum_chromosome_measured = [str(int(qubit.measure())) for qubit in quantum_chromosome]
        new_chromosome = ''.join(quantum_chromosome_measured)
        population.append(new_chromosome)
    
    if dimensions == 1:
        fitness_key = lambda x: fitness(bin_to_float(x))
    else:
        fitness_key = lambda x: fitness(decode_multidim(x, dimensions))
    
    population.sort(key=fitness_key, reverse=maximize)
    
    # if dimensions == 1:
    #     for indiv in population:
    #         print(bin_to_float(indiv))
    # else:
    #     for indiv in population:
    #         print(decode_multidim(indiv, dimensions))
    
    queen = population[0]
    males = population[1:num_males]
    # remaining = population[num_males:]
    # elite = population[:num_elites]
    
    for iteration in range(max_iteration):
        for i in range(population_size - num_elites, population_size):
            selected_male = males[np.random.randint(0, num_males - 1)]
            population[i] = crossover(
                [Qubit(bool(int(digit))) for digit in queen],
                [Qubit(bool(int(digit))) for digit in selected_male],
                crossover_size
            )
        
        population.sort(key=fitness_key, reverse=maximize)
        
        queen = population[0]
        males = population[1:num_males]
        
        if dimensions == 1:
            queen_value = bin_to_float(queen)
            fitness_value = fitness(queen_value)
            print(f'Iteration: {iteration + 1}, Queen: {queen_value}, Fitness: {fitness_value}')
        else:
            queen_values = decode_multidim(queen, dimensions)
            fitness_value = fitness(queen_values)
            print(f'Iteration: {iteration + 1}, Queen: {queen_values}, Fitness: {fitness_value}')
    
    if dimensions == 1:
        return bin_to_float(queen)
    else:
        return decode_multidim(queen, dimensions)

In [298]:
def fitness_first(x: float) -> float:
    return (x - 15) ** 2 

def fitness_second(x: float) -> float:
    if not -5 <= x <= 5:
        return float('inf')  # Penalize out-of-bounds
    
    A = 10
    rastrigin = A + (x**2 - A * math.cos(2 * math.pi * x))
    discontinuity = 50 if -0.1 < x < 0.1 else 0
    noise = np.random.uniform(-0.5, 0.5)
    
    return rastrigin + discontinuity + noise

def deceptive_function(x: float) -> float:

    if not -10 <= x <= 10:
        return float('inf')  # Penalize out-of-bounds
    
    schwefel = 418.9829 - x * math.sin(math.sqrt(abs(x)))
    
    plateau = 100 if -2 <= x <= 2 else 0
    dip = -50 if -0.1 < x < 0.1 else 0
    poly = 0.1 * x**4 - 2 * x**2
    
    return schwefel + plateau + dip + poly


In [299]:
evolution(1000, fitness=deceptive_function, dimensions=1)

Iteration: 1, Queen: 3.255859375, Fitness: 405.75
Iteration: 2, Queen: 3.255859375, Fitness: 405.75
Iteration: 3, Queen: 3.255859375, Fitness: 405.75
Iteration: 4, Queen: 3.255859375, Fitness: 405.75
Iteration: 5, Queen: 3.255859375, Fitness: 405.75
Iteration: 6, Queen: 3.255859375, Fitness: 405.75
Iteration: 7, Queen: 3.255859375, Fitness: 405.75
Iteration: 8, Queen: 3.255859375, Fitness: 405.75
Iteration: 9, Queen: 3.255859375, Fitness: 405.75
Iteration: 10, Queen: 3.255859375, Fitness: 405.75
Iteration: 11, Queen: 3.255859375, Fitness: 405.75
Iteration: 12, Queen: 3.255859375, Fitness: 405.75
Iteration: 13, Queen: 3.255859375, Fitness: 405.75
Iteration: 14, Queen: 3.255859375, Fitness: 405.75
Iteration: 15, Queen: 3.255859375, Fitness: 405.75
Iteration: 16, Queen: 3.255859375, Fitness: 405.75
Iteration: 17, Queen: 3.255859375, Fitness: 405.75
Iteration: 18, Queen: 3.255859375, Fitness: 405.75
Iteration: 19, Queen: 3.255859375, Fitness: 405.75
Iteration: 20, Queen: 3.255859375, Fitne

np.float16(3.256)

In [300]:
def simple_2d_circle(list_x: List[float]) -> float:
    return np.pow(list_x[0] + 5, 2) + np.pow(list_x[1] + 10, 2)
evolution(1000, fitness=simple_2d_circle, dimensions=2, maximize=False)

Iteration: 1, Queen: [np.float16(-0.1504), np.float16(-5.223)], Fitness: 46.375
Iteration: 2, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 3, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 4, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 5, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 6, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 7, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 8, Queen: [np.float16(-0.1504), np.float16(-5.848)], Fitness: 40.75
Iteration: 9, Queen: [np.float16(-0.2441), np.float16(-5.848)], Fitness: 39.875
Iteration: 10, Queen: [np.float16(-0.2441), np.float16(-5.848)], Fitness: 39.875


C:\Users\Dinar\AppData\Local\Temp\ipykernel_18380\336271102.py:2: RuntimeWarning: invalid value encountered in scalar add
  return np.pow(list_x[0] + 5, 2) + np.pow(list_x[1] + 10, 2)
C:\Users\Dinar\AppData\Local\Temp\ipykernel_18380\336271102.py:2: RuntimeWarning: overflow encountered in power
  return np.pow(list_x[0] + 5, 2) + np.pow(list_x[1] + 10, 2)
C:\Users\Dinar\AppData\Local\Temp\ipykernel_18380\336271102.py:2: RuntimeWarning: overflow encountered in scalar add
  return np.pow(list_x[0] + 5, 2) + np.pow(list_x[1] + 10, 2)


Iteration: 11, Queen: [np.float16(-3.906), np.float16(-5.848)], Fitness: 18.4375
Iteration: 12, Queen: [np.float16(-3.906), np.float16(-5.94)], Fitness: 17.671875
Iteration: 13, Queen: [np.float16(-3.906), np.float16(-5.996)], Fitness: 17.234375
Iteration: 14, Queen: [np.float16(-4.812), np.float16(-5.996)], Fitness: 16.0625
Iteration: 15, Queen: [np.float16(-4.812), np.float16(-7.996)], Fitness: 4.05078125
Iteration: 16, Queen: [np.float16(-4.812), np.float16(-11.99)], Fitness: 4.00390625
Iteration: 17, Queen: [np.float16(-4.812), np.float16(-11.99)], Fitness: 4.00390625
Iteration: 18, Queen: [np.float16(-4.812), np.float16(-10.49)], Fitness: 0.27734375
Iteration: 19, Queen: [np.float16(-4.812), np.float16(-10.38)], Fitness: 0.181640625
Iteration: 20, Queen: [np.float16(-4.875), np.float16(-10.38)], Fitness: 0.162109375
Iteration: 21, Queen: [np.float16(-4.875), np.float16(-10.35)], Fitness: 0.13916015625
Iteration: 22, Queen: [np.float16(-4.875), np.float16(-10.35)], Fitness: 0.13916

[np.float16(-4.996), np.float16(-10.0)]